In [1]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import KFold

train = pd.read_csv('../input/nlp-getting-started/train.csv')

In [2]:
kf = KFold(n_splits=3, random_state=1001,shuffle=True)
for i, (train_index, val_index) in enumerate(kf.split(train)):
    trn= train.iloc[train_index].reset_index()
    val= train.iloc[val_index].reset_index()
    
trn = trn.drop(columns=['index'])
val = val.drop(columns=['index'])

val.to_csv('sub_val.csv',index=False)
trn.to_csv('sub_train.csv',index=False)

In [3]:
# !rm -r ./autox
!git clone https://github.com/4paradigm/autox.git
!pip install ./autox

Cloning into 'autox'...
remote: Enumerating objects: 2704, done.
remote: Counting objects: 100% (617/617), done.
remote: Compressing objects: 100% (153/153), done.
remote: Total 2704 (delta 483), reused 576 (delta 457), pack-reused 2087
Receiving objects: 100% (2704/2704), 11.87 MiB | 17.91 MiB/s, done.
Resolving deltas: 100% (1727/1727), done.
Processing ./autox
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 4.3 MB/s eta 0:00:00a 0:00:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.9/948.9 KB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.4/325.4 KB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 31.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 14.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for automl-x: filename=automl_x-

In [4]:
from autox.autox_nlp import NLP_feature
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

In [5]:
df_train = pd.read_csv('sub_train.csv')
df_test = pd.read_csv('sub_val.csv')

In [6]:
use_Toknizer=True
emb_mode = 'Bert'# TFIDF / Word2Vec / Glove / FastText / Bert
encode_mode = 'supervise' # unsupervise / supervise
text_columns_name = ['text']
target_column = df_train['target']
candidate_labels=None

In [7]:
nlp = NLP_feature()
# nlp.do_mlm = True
# nlp.mlm_epochs=3
# nlp.model_name = 'microsoft/deberta-v3-base'
nlp.emb_size=100
nlp.n_clusters=20
df = nlp.fit(df_train,
             text_columns_name,
             use_Toknizer,
             emb_mode,
             encode_mode,
             target_column,
             candidate_labels)

Fitting column: text tokenizer


Downloading:   0%|          | 0.00/285 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Fitting column: text bert embedding


Downloading:   0%|          | 0.00/16.9M [00:00<?, ?B/s]

Some weights of the model checkpoint at prajjwal1/bert-tiny were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Saving model checkpoint to ./text_transformer
Configuration saved in ./text_transformer/config.json
Model weights saved in ./text_transformer/pytorch_model.bin
loading configuration file ./text_transformer/config.json
Model config BertConfig {
  "_name_or_path": "./text_transformer",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob"

Fitting column: text encoder


loading configuration file ./text_transformer/config.json
Model config BertConfig {
  "_name_or_path": "./text_transformer",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 128,
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 2,
  "num_hidden_layers": 2,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file ./text_transformer/pytorch_model.bin
Some weights of the model checkpoint at ./text_transformer were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', '

In [8]:
for column in df.columns:
    df_train[column] = df[column]
df_train = df_train.drop(columns=text_columns_name)

In [9]:
test = nlp.transform(df_test)

loading configuration file ./text_transformer/config.json
Model config BertConfig {
  "_name_or_path": "./text_transformer",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 128,
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 2,
  "num_hidden_layers": 2,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file ./text_transformer/pytorch_model.bin
Some weights of the model checkpoint at ./text_transformer were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', '

Transforming column: text


In [10]:
for column in test.columns:
    df_test[column] = test[column]
df_test = df_test.drop(columns=text_columns_name)

In [11]:
df_train.to_csv(f'{emb_mode}_{encode_mode}_autox_trn.csv',index=False)
df_test.to_csv(f'{emb_mode}_{encode_mode}_autox_val.csv',index=False)

In [12]:
df_val=pd.read_csv(f'{emb_mode}_{encode_mode}_autox_val.csv').drop(columns=['target'])
df_val.to_csv(f'{emb_mode}_{encode_mode}_autox_tst.csv',index=False)

In [13]:
from autox import AutoX

path = f'.' 
autox = AutoX(target = 'target', train_name = f'{emb_mode}_{encode_mode}_autox_trn.csv', test_name = f'{emb_mode}_{encode_mode}_autox_tst.csv',  id = ['id'], path = path)
sub = autox.get_submit()

   INFO ->  [+] read sub_val.csv
   INFO ->  Memory usage of dataframe is 0.10 MB
   INFO ->  Memory usage after optimization is: 0.15 MB
   INFO ->  Decreased by -59.8%
   INFO ->  table = sub_val.csv, shape = (2537, 5)
   INFO ->  [+] read Bert_supervise_autox_tst.csv
   INFO ->  Memory usage of dataframe is 0.08 MB
   INFO ->  Memory usage after optimization is: 0.07 MB
   INFO ->  Decreased by 9.5%
   INFO ->  table = Bert_supervise_autox_tst.csv, shape = (2537, 4)
   INFO ->  [+] read sub_train.csv
   INFO ->  Memory usage of dataframe is 0.19 MB
   INFO ->  Memory usage after optimization is: 0.30 MB
   INFO ->  Decreased by -54.3%
   INFO ->  table = sub_train.csv, shape = (5076, 5)
   INFO ->  [+] read Bert_supervise_autox_trn.csv
   INFO ->  Memory usage of dataframe is 0.19 MB
   INFO ->  Memory usage after optimization is: 0.13 MB
   INFO ->  Decreased by 30.5%
   INFO ->  table = Bert_supervise_autox_trn.csv, shape = (5076, 5)
   INFO ->  [+] read Bert_supervise_autox_val.c

Training on fold 1
Training until validation scores don't improve for 150 rounds
[100]	training's auc: 0.892195	valid_1's auc: 0.844218
[200]	training's auc: 0.90213	valid_1's auc: 0.844091
Early stopping, best iteration is:
[68]	training's auc: 0.889497	valid_1's auc: 0.845478
AUC: 0.8454781836631151
Fold 1 finished in 0:00:00.455650
Training on fold 2
Training until validation scores don't improve for 150 rounds
[100]	training's auc: 0.888237	valid_1's auc: 0.863709
[200]	training's auc: 0.898949	valid_1's auc: 0.864454
[300]	training's auc: 0.909428	valid_1's auc: 0.863854
Early stopping, best iteration is:
[199]	training's auc: 0.898871	valid_1's auc: 0.864553
AUC: 0.864552590266876
Fold 2 finished in 0:00:00.663597
Training on fold 3
Training until validation scores don't improve for 150 rounds
[100]	training's auc: 0.89626	valid_1's auc: 0.827471
[200]	training's auc: 0.906524	valid_1's auc: 0.828983
[300]	training's auc: 0.917246	valid_1's auc: 0.828651
[400]	training's auc: 0.9

   INFO ->  feature importance
   INFO ->                                    feature  fold_1  fold_2  fold_3  fold_4  \
0                       text_meta_feature     276     693     990     866   
1        keyword__text_meta_feature__mean     179     495     768     537   
2      keyword__text_meta_feature__median     125     409     543     465   
3         keyword__text_meta_feature__std     118     281     529     283   
4         keyword__text_meta_feature__min      81     273     562     358   
5         keyword__text_meta_feature__max     107     233     518     283   
6    keyword__text_meta_feature__shift__1      41     195     457     311   
7     keyword__text_meta_feature__diff__2      55     156     402     240   
8     keyword__text_meta_feature__diff__1      45     166     403     241   
9    keyword__text_meta_feature__diff__-3      78     157     395     230   
10    keyword__text_meta_feature__diff__3      58     191     307     237   
11  keyword__text_meta_feature__s

[300]	training's auc: 0.910134	valid_1's auc: 0.854819
Early stopping, best iteration is:
[219]	training's auc: 0.903073	valid_1's auc: 0.855008
AUC: 0.8550075621514321
Fold 5 finished in 0:00:00.752540
Training on fold 1
AUC: 0.832623106060606
Fold 1 finished in 0:00:26.269679
Training on fold 2
AUC: 0.8695001207437817
Fold 2 finished in 0:00:24.650367
Training on fold 3
AUC: 0.8519242593848044
Fold 3 finished in 0:00:24.435282
Training on fold 4
AUC: 0.8517622614008157
Fold 4 finished in 0:00:24.482831
Training on fold 5
AUC: 0.8591003591003591
Fold 5 finished in 0:00:24.430619
Training on fold 6
AUC: 0.8478044871794872
Fold 6 finished in 0:00:24.837828
Training on fold 7
AUC: 0.8613715552739944
Fold 7 finished in 0:00:24.808846
Training on fold 8
AUC: 0.8350846004888733
Fold 8 finished in 0:00:24.513068
Training on fold 9
AUC: 0.8337787959781087
Fold 9 finished in 0:00:25.618585
Training on fold 10


   INFO ->  Average KFold AUC: 0.8449988695208963


AUC: 0.8070391495981333
Fold 10 finished in 0:00:25.343297


In [14]:
val = pd.read_csv(f'sub_val.csv')
from sklearn.metrics import mean_squared_error,roc_auc_score
AUC = (roc_auc_score(val['target'], sub['target']))

In [15]:
AUC

0.8551436007488242